In [1]:
import os

In [2]:
api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.tools.tool_context import ToolContext
from google.genai import types

In [4]:
# Helper functions for running a session
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=7, # Delay multiplier
    initial_delay=1, # initial delay before first retry (seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [11]:
APP_NAME = "Wellness Suite"
USER_ID = "Hector"
SESSION = "default"
USER_PROFILE = {
    "name": USER_ID,
    "age": None,
    "weight": None,
    "height": None,
    "goals": [],
    "injuries": [],
    "dietary_restrictions": []
}

MODEL_NAME = "gemini-2.5-flash-lite"

In [8]:
# Helper function for profile formatting for agent

def get_profile_context():
    """Helper to format the profile for the agents."""
    return f"""
    === USER PROFILE ===
    Name: {USER_PROFILE['name']}
    Stats: {USER_PROFILE['age']} years old, {USER_PROFILE['height']}, {USER_PROFILE['weight']}
    Goals: {', '.join(USER_PROFILE['goals'])}
    Injuries: {', '.join(USER_PROFILE['injuries'])}
    Diet: {', '.join(USER_PROFILE['dietary_restrictions'])}     
    """

In [9]:
def update_user_profile(attribute: str, value: str):
    """
    Updates the user's profile with new information.
    Use this when the user explicitly changes a preference, stat, or goal.

    Args:
        attribute: The specific field to update (e.g. 'weight', 'dietary_restriction', 'goals').
        value: The new value for that attribute.
    """
    # Clean the input slightly to handle list-like strings
    if attribute in ["goals", "injuries", "dietary_restrictions"]:
        # Simple logic to append to list if list type
        if isinstance(USER_PROFILE.get(attribute), list):
            if value not in USER_PROFILE[attribute]:
                USER_PROFILE[attribute].append(value)
                return f"Updated {attribute}: Added '{value}'."
            return f"{attribute} already contains '{value}'."
        
    # Default: Overwrite the value
    old_val = USER_PROFILE.get(attribute, "N/A")
    USER_PROFILE[attribute] = value
    print(f"\n[System] Memory Updated: {attribute} changed from '{old_val}' to '{value}'.")
    return f"Successfully updated {attribute} to {value}."

In [12]:
current_user_context = get_profile_context()

In [13]:
# Create Agents

# Fitness trainer agent
fitness_trainer_agent = Agent(
    name="FitnessTrainerAgent",
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=f"""
You are 'Apex', an elite strength and conditioning coach.
Your goal is to provide safe, actionable, and scientifically backed workout advice.

{get_profile_context()}

- Always ask about injuries if the user suggests a high-intensity movement.
- Focus on form cues and progressive overload.
- If the query is vague, provide a general framework but ask clarifying questions.
- Keep your tone motivating but disciplined.
""",
    tools=[],
    output_key="fitness_plan",
)

# Nutrition advisor agent
nutrition_advisor_agent = Agent(
    name="NutritionAdvisorAgent",
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=f"""
You are 'Nourish', a clinical nutritionist and dietician.
Your goal is to help users fuel their bodies for their specific goals.

{get_profile_context()}

- Focus on whole foods, macronutrient balance, and sustainability.
- Do not prescribe medical treatments; always advise consulting a doctor for medical issues.
- When discussing weight loss/gain, emphasize caloric balance and protein intake.
- Keep your tone empathetic and educational.
""",
    tools=[],
    output_key="nutrition_plan",
)

# Coordinator root agent
root_agent = Agent(
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    name="WellnessCoordinatorAgent",
    description="A holistic wellness coordinator agent",
    instruction=f"""
You are the 'Wellness Coordinator'. You are the primary interface for the user.
Your job is to understand the user's holistic wellness goals and coordinate advice.

{get_profile_context()}

1. If the user explicitly provides new personal data (e.g. "I hurt my back", "I am now vegan", "My weight is now 175"), use the `update_user_profile` tool to update the user's data. After updating, confirm the change to the user.
2. If the user asks a general question, answer it yourself.
3. If the user asks about workouts/exercise, use the `fitness_trainer_agent` as an `AgentTool`.
4. If the user asks about diet/food, use the 'nutrition_advisor_agent' as an `AgentTool`.
5. If the user asks a complex question involving BOTH (e.g., "How to lose weight"),
   you may need to call BOTH tools sequentially to get a full picture.

ALWAYS synthesize the information returned by the tools into a cohesive, friendly response.
Do not just paste the tool output; summarize it and make it personal to the user.
"""
,
    tools=[
        update_user_profile,
        AgentTool(fitness_trainer_agent),
        AgentTool(nutrition_advisor_agent)
    ],
)

In [14]:
# Set up Session Management
# InMemorySessionService stores conversations in RAM

session_service = InMemorySessionService()

In [15]:
# Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("Stateful agent initialized")
print(f" - Application: {APP_NAME}")
print(f" - User: {USER_ID}")
print(f" - Using: {session_service.__class__.__name__}")

Stateful agent initialized
 - Application: Wellness Suite
 - User: Hector
 - Using: InMemorySessionService


In [16]:
await run_session(
    runner,
    [
        "Hello!"
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hello!
gemini-2.5-flash-lite >  Hello Hector! How can I help you prioritize your well-being today?


In [17]:
await run_session(
    runner,
    [
        "Do you know how much I weigh?"
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Do you know how much I weigh?
gemini-2.5-flash-lite >  I do not have information about your current weight. If you'd like to share it, I can update your profile. 



In [18]:
await run_session(
    runner,
    [
        "I weigh 170 lbs."
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > I weigh 170 lbs.



[System] Memory Updated: weight changed from 'None' to '170 lbs'.
gemini-2.5-flash-lite >  I've updated your profile to reflect that you weigh 170 lbs.


In [19]:
await run_session(
    runner,
    [
        "How much do I weigh?"
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > How much do I weigh?
gemini-2.5-flash-lite >  You currently weigh 170 lbs.


In [20]:
await run_session(
    runner,
    [
        "My goal is to lose 20 lbs over the next year."
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > My goal is to lose 20 lbs over the next year.


gemini-2.5-flash-lite >  That's a great goal, Hector! I've added "Lose 20 lbs in the next year" to your wellness objectives.

To help you achieve this, would you like to explore some workout routines or discuss dietary changes?


In [21]:
await run_session(
    runner,
    [
        "Sure, I would like to explore both. Let me know if there is any other information about me you need to help better."
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Sure, I would like to explore both. Let me know if there is any other information about me you need to help better.


gemini-2.5-flash-lite >  It's great that you're looking to approach your goal from both exercise and diet, Hector! This is a fantastic way to ensure sustainable and healthy progress.

The **Fitness Trainer Agent** has outlined a solid strategy for incorporating strength training and cardio to help you lose 20 lbs over the next year. To tailor this perfectly for you, they need a bit more information:

*   **Injuries/Limitations:** Do you have any current or past injuries, medical conditions, or physical limitations?
*   **Fitness Level:** What's your current experience with exercise (beginner, intermediate, advanced)?
*   **Equipment Access:** What equipment do you have available (gym, home gym, bodyweight only)?
*   **Time Commitment:** How many days per week and how long per session can you dedicate to working out?

The **Nutrition Advisor Agent** has provided a comprehensive guide for a healthy eating plan to support your goal. They've emphasized:

*   **Prioritizing Protein:** Cruci

In [22]:
await run_session(
    runner,
    [
        "I tore my left rotator cuff and had it surgically repaired about 6 years ago. I am currently at a beginner level of fitness, but was at an advanced level in graduate school. I have access to a gym, and would be willing to work out about 3 times a week."
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > I tore my left rotator cuff and had it surgically repaired about 6 years ago. I am currently at a beginner level of fitness, but was at an advanced level in graduate school. I have access to a gym, and would be willing to work out about 3 times a week.


gemini-2.5-flash-lite >  Thanks for providing that information, Hector! Knowing about your past rotator cuff surgery is very important for designing a safe and effective workout plan.

The **Fitness Trainer Agent** can now create a plan that avoids aggravating your shoulder while still helping you reach your goals. They'll focus on exercises that are safe for your repaired rotator cuff and suitable for a beginner's level, while keeping in mind you have gym access and can work out 3 times a week.

The **Nutrition Advisor Agent** has already given you a great framework for healthy eating. Remember, consistency with both your nutrition and your workouts will be key to successfully losing those 20 lbs over the next year.

Would you like me to ask the Fitness Trainer Agent to generate a workout plan based on this information?
gemini-2.5-flash-lite >  This is fantastic, Hector! With the information you've provided, the **Fitness Trainer Agent** has crafted a detailed 3-day-a-week workout pla